In [1]:
#import mllib package since we will use ML models
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.feature import HashingTF

#global
spam = None
ham = None

In [2]:
# local config 
def local_code():
    global spam
    global ham
    sc = None
    os.environ['HADOOP_HOME'] = os.path.abspath(os.path.join(os.getcwd(), os.pardir)).replace('\\', '/')+'/util/hadoop-2.7.1'
    while sc is None: 
        try: 
            sc = SparkContext.getOrCreate(SparkConf().setMaster("local"))
        except: 
            pass
    spam = sc.textFile("spam.txt")
    ham = sc.textFile("ham.txt")
    return sc

In [3]:
# aws config 
def aws_code(): 
    global spam
    global ham
    spam = sc.textFile("s3://minghaohadoophw2/input/spam.txt")
    ham = sc.textFile("s3://minghaohadoophw2/input/ham.txt")

In [4]:
#environment config
try: sc
except NameError: sc = local_code()
else: aws_code()
#initiate a feature vector
tf = HashingTF()
sc

<SparkContext master=local appName=pyspark-shell>

In [5]:
#each email is split into words, each word is mapped to one feature
#thus each email is mapped to a feature vector
spamFeatures = spam.map(lambda email: tf.transform(email.split(" ")))
hamFeatures = ham.map(lambda email: tf.transform(email.split(" ")))
#for ML purpose (classifier), spam features are labeled as positive (1)
#non-spam features are labeled as negative (0)
positiveExamples = spamFeatures.map(lambda features: LabeledPoint(1, features))
negativeExamples = hamFeatures.map(lambda features: LabeledPoint(0, features))
#union spam and ham examples with labels now, for classifier
training_data = positiveExamples.union(negativeExamples)
#cache data since Logistic Regression is an iterative algorithm
training_data.cache()
#use ML model logistic regression with SGD method
model = LogisticRegressionWithSGD.train(training_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#testing: first transform test cases to words (features)
posTestExample = tf.transform("O M G GET cheap stuff by sending money to ...".split(" "))
negTestExample = tf.transform("Hi Dad, I started studying Spark the other ...".split(" "))
#use classifier from training stage to predict the test cases
print ("Prediction for positive test example: %g" % model.predict(posTestExample))
print ("Prediction for negative test example: %g" % model.predict(negTestExample))
#call stop function from SparkContext to clean up resources
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Prediction for positive test example: 1
Prediction for negative test example: 0